# Notebook 05: Volume Analysis

Learn why volume is crucial in confirming price movements and identifying potential reversals.

## What You'll Learn

1. Why volume matters in technical analysis
2. Volume trends and patterns
3. Volume confirmation of price movements
4. On-Balance Volume (OBV) indicator
5. Volume-weighted price analysis
6. Identifying accumulation and distribution

## Time Required: 45-60 minutes

---

In [ ]:
# Setup
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

warnings.filterwarnings("ignore")

# Try importing pandas_ta
try:
    import pandas_ta as ta

    print("✅ pandas-ta imported successfully!")
except ImportError:
    print("⚠️ pandas-ta not found. Install it with:")
    print("   pip install pandas-ta")
    print("\nYou need pandas-ta for volume indicators (OBV).")

# Fetch data
ticker = "0310.KL"  # UUE Holdings Berhad - Malaysian construction & engineering

try:
    stock = yf.Ticker(ticker)
    data = stock.history(period="6mo")

    if len(data) == 0:
        print(f"\n⚠️ No data returned for {ticker}")
        print("💡 Try: 1155.KL (Maybank) or 1295.KL (Public Bank)")
    else:
        # Calculate volume statistics
        data["Volume_MA_20"] = data["Volume"].rolling(window=20).mean()
        data["Price_Change"] = data["Close"].pct_change()
        data["Volume_Ratio"] = data["Volume"] / data["Volume_MA_20"]

        print(f"✅ Data fetched for {ticker} (UUE Holdings Berhad)")
        print(f"\nAverage Daily Volume: {data['Volume'].mean():,.0f}")
        print(f"Latest Volume: {data['Volume'].iloc[-1]:,.0f}")
        print(f"Volume vs Average: {(data['Volume'].iloc[-1] / data['Volume'].mean()):.1f}x")
except Exception as e:
    print(f"\n⚠️ Error: {e}")
    print("Check your internet connection.")

In [ ]:
# Visualize price and volume
if "data" not in locals() or len(data) == 0:
    print("⚠️ Please run the previous cell first to fetch stock data.")
else:
    fig = make_subplots(
        rows=2,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        row_heights=[0.7, 0.3],
        subplot_titles=(f"{ticker} Price", "Volume"),
    )

    # Price
    fig.add_trace(
        go.Scatter(x=data.index, y=data["Close"], name="Price", line=dict(color="blue")),
        row=1,
        col=1,
    )

    # Volume bars (green for up days, red for down days)
    colors = [
        "green" if data["Close"].iloc[i] >= data["Open"].iloc[i] else "red"
        for i in range(len(data))
    ]

    fig.add_trace(
        go.Bar(x=data.index, y=data["Volume"], name="Volume", marker_color=colors), row=2, col=1
    )

    # Volume MA
    fig.add_trace(
        go.Scatter(
            x=data.index,
            y=data["Volume_MA_20"],
            name="Volume MA(20)",
            line=dict(color="orange", dash="dash"),
        ),
        row=2,
        col=1,
    )

    fig.update_layout(height=800, template="plotly_white", hovermode="x unified")
    fig.update_yaxes(title_text="Price (RM)", row=1, col=1)
    fig.update_yaxes(title_text="Volume", row=2, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)

    fig.show()

    print("✅ Price and Volume chart created!")

In [ ]:
# Calculate OBV
if "data" not in locals() or len(data) == 0:
    print("⚠️ Please run the previous cells first to fetch stock data.")
else:
    try:
        data["OBV"] = ta.obv(data["Close"], data["Volume"])

        # Create visualization
        fig = make_subplots(
            rows=2,
            cols=1,
            shared_xaxes=True,
            vertical_spacing=0.05,
            row_heights=[0.5, 0.5],
            subplot_titles=(f"{ticker} Price", "On-Balance Volume (OBV)"),
        )

        # Price
        fig.add_trace(
            go.Scatter(x=data.index, y=data["Close"], name="Price", line=dict(color="blue")),
            row=1,
            col=1,
        )

        # OBV
        fig.add_trace(
            go.Scatter(x=data.index, y=data["OBV"], name="OBV", line=dict(color="purple")),
            row=2,
            col=1,
        )

        fig.update_layout(height=800, template="plotly_white", hovermode="x unified")
        fig.update_yaxes(title_text="Price (RM)", row=1, col=1)
        fig.update_yaxes(title_text="OBV", row=2, col=1)
        fig.update_xaxes(title_text="Date", row=2, col=1)

        fig.show()

        print("✅ OBV chart created!")
        print("\n💡 Look for:")
        print("   - Price and OBV moving together (confirmation)")
        print("   - Price up but OBV down (bearish divergence)")
        print("   - Price down but OBV up (bullish divergence)")
    except ImportError:
        print("⚠️ pandas-ta is required for OBV. Install with: pip install pandas-ta")
    except Exception as e:
        print(f"⚠️ Error calculating OBV: {e}")

In [ ]:
# Find high volume days
if "data" not in locals() or len(data) == 0:
    print("⚠️ Please run the previous cells first to fetch stock data.")
elif "Volume_Ratio" not in data.columns:
    print("⚠️ Please run the first cell to calculate volume ratios.")
else:
    high_volume_days = data[data["Volume_Ratio"] > 2.0].copy()

    print("🔥 High Volume Days (>2x average)\n")
    print("=" * 60)
    print(f"\nFound {len(high_volume_days)} high volume days\n")

    if len(high_volume_days) > 0:
        print("Recent high volume events:")
        for date in high_volume_days.tail(5).index:
            volume = data.loc[date, "Volume"]
            price_change = data.loc[date, "Price_Change"] * 100
            ratio = data.loc[date, "Volume_Ratio"]
            direction = "📈 UP" if price_change > 0 else "📉 DOWN"
            print(f"\n   {date.strftime('%Y-%m-%d')}:")
            print(f"      Volume: {volume:,.0f} ({ratio:.1f}x average)")
            print(f"      Price: {direction} {abs(price_change):.2f}%")

## Part 3: Volume Analysis Patterns

### High Volume Events

Let's identify days with unusually high volume (2x average or more)

## Key Takeaways

### ✅ What You Learned

1. **Volume confirms** price movements
2. **High volume** shows strong conviction
3. **OBV** tracks cumulative buying/selling pressure
4. **Volume patterns** can predict reversals
5. Always **combine volume** with price analysis

### 📚 Next Steps

Notebook 06: Putting It All Together

---

**Happy Learning! 📚📊**